In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import importlib
from scipy.io import FortranFile

import time 
import cftime 
#import ERA5procx as Ex
import GenRegrid as GR

importlib.reload( GR )

## START HERE FOR "BATCH"

In [ ]:
RegridMethod="BILINEAR" 
foo = GR.prep(Dst='fv1x1',DstVgrid='L58',Src='ERAI',WOsrf=True, RegridMethod=RegridMethod )

In [ ]:
#faa=GR.get_ERA5( year=2021,month=6,day=1,hour0=0 )

faa=GR.get_ERAI( year=2017,month=1,day=1 )

In [ ]:
StartTime = time.asctime( time.localtime(time.time()) )
print( StartTime )

skoo =time.strftime("%a, %d %b %Y %H:%M:%S +0000", time.gmtime())
print( skoo )
vers =time.strftime("%Y%m%d_%H%M", time.gmtime())
print( vers )

In [ ]:
#pmid_ERA, pmid_CAM, pmid_CAM_zERA , te_ERA, te_ERA_xCAM , te_ERA_xzCAM  =GR.xRegrid()

ExitAfterWinds=False
ExitAfterTemperature=False
verstime=time.strftime("%Y%m%d_%H%M", time.gmtime() ) + '_'+RegridMethod+'_lnPS'
if ( ExitAfterTemperature == True):
    print( " Exiting after Temp " )
    pmid_ERA, lat_ERA, lon_ERA, te_ERA, \
    pmid_CAM_zERA, lat_CAM, lon_CAM, te_ERA_xCAM, \
    pmid_CAM, te_ERA_xzCAM, \
    ps_ERA, ps_CAM, ps_ERA_xCAM, \
    phis_ERA, phis_CAM, phis_ERA_xCAM = GR.xRegrid( ExitAfterTemperature=True, 
                                                    HorzInterpLnPs=True )
elif ( ExitAfterWinds == True):
    print( " Exiting after Winds " )
    pmid_ERA, lat_ERA, lon_ERA, u_ERA, \
    pmid_CAM_zERA, lat_CAM, lon_CAM, u_ERA_xCAM, \
    pmid_CAM, u_ERA_xzCAM, \
    ps_ERA, ps_CAM, ps_ERA_xCAM, \
    phis_ERA, phis_CAM, phis_ERA_xCAM = GR.xRegrid( ExitAfterWinds=True, 
                                                    HorzInterpLnPs=True )
else:
    print( " Running through and writing netcdf " )
    fii=GR.xRegrid(HorzInterpLnPs=True)
    fuu=GR.write_netcdf(version=verstime )

In [ ]:
#fuu=GR.write_netcdf(version='test02-oldtopo')

In [ ]:
file1='/glade/p/cgd/amp/juliob/ERAI/f09_omega/L58/ERAI_fv09_L58_bilin_teZ.cam2.i.2017-01-01-64800.nc'
file2='/glade/p/cgd/amp/juliob/ERAI/f09_omega/L58/ERAI_fv09_L58_bilin_simple_tbot_teZ.cam2.i.2017-01-01-64800.nc'
ds1=xr.open_dataset( file1, decode_times=False)
ds2=xr.open_dataset( file2, decode_times=False)


print( list(ds1.variables) )

q1=ds1['Q'].values
u1=ds1['U'].values
u2=ds2['U'].values
t1=ds1['T'].values
t2=ds2['T'].values
ps1=ds1['PS'].values
ps2=ds2['PS'].values
phis1=ds1['PHIS'].values
phis1_ERA=ds1['PHIS_input'].values
hybm=ds1['hybm'].values
hyam=ds1['hyam'].values
lono=ds1['lon'].values
lato=ds1['lat'].values
lolo=ps1[0,124,72]*hybm+100_000.*hyam

#plt.plot( t1.flatten())


In [ ]:
file3='/glade/scratch/juliob/ERAI_x_fv1x1_L58_20230331_2012_BILINEAR_lnPS_WO.2017-01-01-64800.nc'
ds3=xr.open_dataset( file3, decode_times=False)
t3=ds3['T'].values



In [ ]:
# Read Fortran unf file from within MAKEIC_psadj
# Corresponds to 2017-01-01-64800
#-----------------------------------------------
FileN = '/glade/work/juliob/Nudging/Gen_Data/Gen_Data_ERAI_f09-STUB/MAKEIC_psadj_2017-01-01-64800.dat'

f=FortranFile( FileN, 'r')
idims=f.read_record( '<i4' )

print(idims)
plev=idims[0]
plat=idims[1]
plon=idims[2]
phis_new = f.read_record( '<f8'   ).reshape(plat,plon)
phis_old = f.read_record( '<f8'   ).reshape(plat,plon)
ps_new = f.read_record( '<f8'   ).reshape(plat,plon)
ps_old = f.read_record( '<f8'   ).reshape(plat,plon)
te_ERA_xCAM_wo = f.read_record( '<f8'   ).reshape(plev,plat,plon)
print(np.shape( te_ERA_xCAM_wo ) )

In [ ]:
# Read Fortran unf file from within MAKEIC_psadj
# Corresponds to 2017-01-01-64800
#-----------------------------------------------
FileN = '/glade/work/juliob/Nudging/Gen_Data/Gen_Data_ERAI_f09-STUB/MAKEIC_vert_quad_opt1_2017-01-01-64800.dat'

f=FortranFile( FileN, 'r')
idims=f.read_record( '<i4' )

print(idims)
plev=idims[0]
plevi=idims[1]
plat=idims[2]
plon=idims[3]
print(  plev,plevi,plat,plon )
press_ERA        = f.read_record( '<f8'   ).reshape(plat,plon,plevi).transpose(2,0,1)
press_CAM        = f.read_record( '<f8'   ).reshape(plat,plon,plev).transpose(2,0,1)
te_ERA_xCAM_wo2  = f.read_record( '<f8'   ).reshape(plat,plon,plevi).transpose(2,0,1)
te_ERA_xzCAM_wo2 = f.read_record( '<f8'   ).reshape(plat,plon,plev).transpose(2,0,1)
#ps_old = f.read_record( '<f8'   ).reshape(plat,plon)
#te_WO_ERA_xCAM = f.read_record( '<f8'   ).reshape(plev,plat,plon)
print(np.shape( press_ERA ) )
#plt.contour(te_ERA_xCAM_wo2[59,:,:] - te_ERA_xCAM_wo[59,:,:] )
#plt.plot( (te_ERA_xCAM_wo2[59,:,:] - te_ERA_xCAM_wo[59,:,:] ).flatten() )
#plt.plot( (te_ERA_xCAM[3,59,:,:] - te_ERA_xCAM_wo[59,:,:] ).flatten() )

Issue is that 'zERA' in Python code is being defined using ps_CAM, while in W&O 'zERA' is being defined with 'ps_old' (aka ps_ERA_xCAM in Python code )

In [ ]:
clev=np.linspace(200,330,num=31)
dlev=np.linspace(-10,10,num=21)
#clev=np.linspace(277,287,num=31)
#ERA - 89 128
#CAM -124 72
print(lon_ERA[128], lat_ERA[89])

# Global
elon,wlon=0,360
slat,nlat=-90,90

# Tibet ...
#elon,wlon=70,100
#slat,nlat=25,40

# Andes ...
#elon,wlon=280,300
#slat,nlat=-40,-10

# Himalayan Front
#elon,wlon=85,95
#slat,nlat=25,28


kCAM=40
kERA=59
nCAM=3

fig,ax = plt.subplots(2,2,figsize=(18,18) ,squeeze=False )

i,j=0,0
cf = ax[i,j].contourf( lon_CAM, lat_CAM, te_ERA_xzCAM[nCAM,kCAM,:,:],levels=clev )
ax[i,j].set_xlim(elon,wlon)
ax[i,j].set_ylim(slat,nlat)
ax[i,j].set_title("ERA Temp on xzCAM grid from Python code")
plt.colorbar(cf,ax=ax[i,j] )

i,j=0,1
cf = ax[i,j].contourf( lon_CAM, lat_CAM, t1[0,kCAM,:,:],levels=clev )
ax[i,j].set_xlim(elon,wlon)
ax[i,j].set_ylim(slat,nlat)
ax[i,j].set_title("ERA Temp on xzCAM grid from GoW&O ")
plt.colorbar(cf,ax=ax[i,j] )


diff_te = te_ERA_xzCAM[nCAM,kCAM,:,:] - t3[kCAM,:,:]
#diff_te = t2[0,kCAM,:,:] - t1[0,kCAM,:,:]

i,j=1,0
cf = ax[i,j].contourf( lon_CAM, lat_CAM, diff_te ,levels=dlev )
ax[i,j].set_xlim(elon,wlon)
ax[i,j].set_ylim(slat,nlat)
#ax[i,j].set_title("ERA Temp on CAM Z-grid and CAM H-grid, but from W&O regridder")
ax[i,j].set_title(" Diff between Here and W&O ")
plt.colorbar(cf,ax=ax[i,j] )


In [ ]:
# Let's look at some interensitng points here
#
location="Himalayas"
location="Andes"
nCAM=3

if (location=="Himalayas"):
    #Himalayan peaks
    ilon=np.argmin( np.abs( lon_CAM - 76. ))
    ilat=np.argmin( np.abs( lat_CAM - 36. ))
    ylim=(60_000.,42_000.)
    xlim=(-10,10 )


if (location=="Andes"):
    #Andean peaks
    ilon=np.argmin( np.abs( lon_CAM - 290. ))
    ilat=np.argmin( np.abs( lat_CAM - (-30.) ))
    ylim=(75_000.,50_000.)
    xlim=(-10,10 )

if (location=="Australia"):
    #Australia desert (Alice Springs)
    ilon=np.argmin( np.abs( lon_CAM - 135. ))
    ilat=np.argmin( np.abs( lat_CAM - (-23.4) ))
    ylim=(96_000.,85_000.)
    xlim=(-10,10 )


#print(te_ERA_xzCAM[nCAM,kCAM,ilat,ilon] )



jCAM,iCAM=ilat,ilon
"""
plt.plot( u_ERA_xCAM [nCAM,:, jCAM, iCAM] , pmid_CAM_zERA [nCAM,:, jCAM, iCAM], '-x' 
         , label='ERA U from Python code on xCAM and zERA' )


plt.plot( u_ERA_xzCAM [nCAM,:, jCAM, iCAM] , pmid_CAM [nCAM,:, jCAM, iCAM], '-^' 
         , label='ERA U from Python code on xzCAM' )
"""
plt.plot( q1 [0,:, jCAM, iCAM] , pmid_CAM [nCAM,:, jCAM, iCAM], '-^' 
         , label='ERA U from W&O code' )


"""
plt.plot( te_ERA_xzCAM_wo2[:, jCAM, iCAM]+2 , press_CAM[:, jCAM, iCAM], '-*' 
         , label='ERA Temp on xzCAM from GUTS OF W&O' )
"""

plt.ylim( ylim )
#plt.xlim( xlim )


plt.legend(loc='upper left')


In [ ]:
# Let's look at some interensitng points here
#
location="Himalayas"
location="Andes"
nCAM=3

if (location=="Himalayas"):
    #Himalayan peaks
    ilon=np.argmin( np.abs( lon_CAM - 76. ))
    ilat=np.argmin( np.abs( lat_CAM - 36. ))
    ylim=(60_000.,42_000.)
    xlim=(-10,10 )


if (location=="Andes"):
    #Andean peaks
    ilon=np.argmin( np.abs( lon_CAM - 290. ))
    ilat=np.argmin( np.abs( lat_CAM - (-30.) ))
    ylim=(75_000.,50_000.)
    xlim=(-10,10 )

if (location=="Australia"):
    #Australia desert (Alice Springs)
    ilon=np.argmin( np.abs( lon_CAM - 135. ))
    ilat=np.argmin( np.abs( lat_CAM - (-23.4) ))
    ylim=(96_000.,85_000.)
    xlim=(-10,10 )


#print(te_ERA_xzCAM[nCAM,kCAM,ilat,ilon] )



jCAM,iCAM=ilat,ilon
plt.plot( u_ERA_xCAM [nCAM,:, jCAM, iCAM] , pmid_CAM_zERA [nCAM,:, jCAM, iCAM], '-x' 
         , label='ERA U from Python code on xCAM and zERA' )


plt.plot( u_ERA_xzCAM [nCAM,:, jCAM, iCAM] , pmid_CAM [nCAM,:, jCAM, iCAM], '-^' 
         , label='ERA U from Python code on xzCAM' )

plt.plot( u1 [0,:, jCAM, iCAM] , pmid_CAM [nCAM,:, jCAM, iCAM], '-^' 
         , label='ERA U from W&O code' )


"""
plt.plot( te_ERA_xzCAM_wo2[:, jCAM, iCAM]+2 , press_CAM[:, jCAM, iCAM], '-*' 
         , label='ERA Temp on xzCAM from GUTS OF W&O' )
"""

plt.ylim( ylim )
plt.xlim( xlim )


plt.legend(loc='upper left')


In [ ]:
# Let's look at some interensitng points here
#
location="Himalayas"
location="Andes"

if (location=="Himalayas"):
    #Himalayan peaks
    ilon=np.argmin( np.abs( lon_CAM - 76. ))
    ilat=np.argmin( np.abs( lat_CAM - 36. ))
    ylim=(60_000.,42_000.)
    xlim=(240,270 )


if (location=="Andes"):
    #Andean peaks
    ilon=np.argmin( np.abs( lon_CAM - 290. ))
    ilat=np.argmin( np.abs( lat_CAM - (-30.) ))
    ylim=(75_000.,60_000.)
    xlim=(280,300 )

if (location=="Australia"):
    #Australia desert (Alice Springs)
    ilon=np.argmin( np.abs( lon_CAM - 135. ))
    ilat=np.argmin( np.abs( lat_CAM - (-23.4) ))
    ylim=(96_000.,85_000.)
    xlim=(280,320 )


#print(te_ERA_xzCAM[nCAM,kCAM,ilat,ilon] )



jCAM,iCAM=ilat,ilon
plt.plot( te_ERA_xCAM [nCAM,:, jCAM, iCAM] , pmid_CAM_zERA [nCAM,:, jCAM, iCAM], '-x' 
         , label='ERA Temp from Python code on xCAM and zERA' )

"""
plt.plot( te_ERA_xCAM_wo2[:, jCAM, iCAM]-0 , press_ERA[:, jCAM, iCAM], '-o' 
         , label='ERA Temp on xCAM and zERA from GUTS OF W&O' )
"""

plt.plot( te_ERA_xzCAM [nCAM,:, jCAM, iCAM] , pmid_CAM [nCAM,:, jCAM, iCAM], '-^' 
         , label='ERA Temp from Python code on xzCAM' )

plt.plot( t1 [0,:, jCAM, iCAM] , pmid_CAM [nCAM,:, jCAM, iCAM], '-^' 
         , label='ERA Temp from W&O code' )


"""
plt.plot( te_ERA_xzCAM_wo2[:, jCAM, iCAM]+2 , press_CAM[:, jCAM, iCAM], '-*' 
         , label='ERA Temp on xzCAM from GUTS OF W&O' )
"""

plt.ylim( ylim )
plt.xlim( xlim )


plt.legend(loc='upper left')


In [ ]:
print( np.minimum( [0.],[200])[0] )

In [ ]:
Dphis = phis_ERA_xCAM - phis_CAM
plt.plot(Dphis.flatten()-dphis_WO.flatten() )

## Check Mapping files

In [ ]:


#ERA - 89 128
#CAM -124 72
jCAM,iCAM = 124,72
jERA,iERA = 89,128
nCAM = 3

plt.plot( te_ERA_xCAM [nCAM,:, jCAM, iCAM] , pmid_CAM_zERA [nCAM,:, jCAM, iCAM], '-x' 
         , label='ERA Temp from Python code on xCAM and zERA' )

plt.plot( te_ERA_xCAM_wo2[:, jCAM, iCAM] , press_ERA[:, jCAM, iCAM], '-o' 
         , label='ERA Temp on xCAM and zERA from GUTS OF W&O' )



#plt.plot( te_ERA_xzCAM [0,:,124, 72] , pmid_CAM [0,:,124, 72], '-o' , label='ERA Temp from Python code on xzCAM' )
#plt.plot( te_ERA[0,:,89, 128] ,pmid_ERA [0,:,89, 128] ,'-+', label='ERA Temp at closest xERA point' )


plt.ylim(88_000.,70_000.)
plt.xlim(270,290)
plt.legend()

print(ps_new[jCAM,iCAM])
print(ps_CAM[nCAM,jCAM,iCAM])

print(ps_old[jCAM,iCAM])
print(ps_ERA_xCAM[nCAM,jCAM,iCAM])



In [ ]:


#ERA - 89 128
#CAM -124 72
jCAM,iCAM = 124,72
jERA,iERA = 89,128
nCAM = 3

plt.plot( te_ERA_xCAM [nCAM,:, jCAM, iCAM] , pmid_CAM_zERA [nCAM,:, jCAM, iCAM], '-x' 
         , label='ERA Temp from Python code on xCAM and zERA' )

#plt.plot( te_ERA_xCAM_wo2[:, jCAM, iCAM] , press_ERA[:, jCAM, iCAM], '-o' 
#         , label='ERA Temp on xCAM and zERA from GUTS OF W&O' )


plt.plot( te_ERA_xzCAM [nCAM,:, jCAM, iCAM] , pmid_CAM [nCAM,:, jCAM, iCAM], '-^' 
         , label='ERA Temp from Python code on xzCAM' )

plt.plot( t1[0,:, jCAM, iCAM]+1 , pmid_CAM[nCAM,:, jCAM, iCAM], '-*' 
         , label='ERA Temp on xzCAM from W&O' )

plt.plot( t2[0,:, jCAM, iCAM]+.5 , pmid_CAM[nCAM,:, jCAM, iCAM], '-*' 
         , label='ERA Temp on xzCAM from W&O (simple TBOT)' )



#plt.plot( te_ERA_xzCAM [0,:,124, 72] , pmid_CAM [0,:,124, 72], '-o' , label='ERA Temp from Python code on xzCAM' )
#plt.plot( te_ERA[0,:,89, 128] ,pmid_ERA [0,:,89, 128] ,'-+', label='ERA Temp at closest xERA point' )


plt.ylim(88_000.,70_000.)
plt.xlim(270,290)
plt.legend()

#print(ps_new[jCAM,iCAM])
#print(ps_CAM[nCAM,jCAM,iCAM])

#print(ps_old[jCAM,iCAM])
#print(ps_ERA_xCAM[nCAM,jCAM,iCAM])

